In [16]:
# PennyLane imports
import pennylane as qml
from pennylane import numpy as pnp
from pennylane.optimize import GradientDescentOptimizer, AdamOptimizer
from pennylane.templates import BasicEntanglerLayers, StronglyEntanglingLayers

# General imports
import numpy as np

from qiskit.quantum_info import SparsePauliOp

In [2]:
def create_matrix(cut_off, type):
    # Initialize a zero matrix of the specified size
    matrix = np.zeros((cut_off, cut_off), dtype=np.complex128)
    
    # Fill the off-diagonal values with square roots of integers
    for i in range(cut_off):
        if i > 0:  # Fill left off-diagonal
            if type == 'q':
                matrix[i][i - 1] = (1/np.sqrt(2)) * np.sqrt(i)  # sqrt(i) for left off-diagonal
            else:
                matrix[i][i - 1] = (1j/np.sqrt(2)) * np.sqrt(i)

        if i < cut_off - 1:  # Fill right off-diagonal
            if type == 'q':
                matrix[i][i + 1] = (1/np.sqrt(2)) * np.sqrt(i + 1)  # sqrt(i + 1) for right off-diagonal
            else:
                matrix[i][i + 1] = (-1j/np.sqrt(2)) * np.sqrt(i + 1)

    return matrix


# Function to calculate the Hamiltonian
def calculate_Hamiltonian(cut_off, potential):
    # Generate the position (q) and momentum (p) matrices
    q = create_matrix(cut_off, 'q')  # q matrix
    p = create_matrix(cut_off, 'p')  # p matrix

    # Calculate q^2 and q^3 for potential terms
    q2 = np.matmul(q, q)
    q3 = np.matmul(q2, q)

    #fermionic identity
    I_f = np.eye(2)

    #bosonic identity
    I_b = np.eye(cut_off)

    # Superpotential derivatives
    if potential == 'QHO':
        W_prime = q  # W'(q) = q
        W_double_prime = I_b #W''(q) = 1

    elif potential == 'AHO':
        W_prime = q + q3  # W'(q) = q + q^3
        W_double_prime = I_b + 3 * q2  # W''(q) = 1 + 3q^2

    elif potential == 'DW':
        W_prime = q + q2 + I_b  # W'(q) = q + q^2 + 1
        W_double_prime = I_b + 2 * q  # W''(q) = 1 + 2q

    else:
        print("Not a valid potential")
        raise

    # Kinetic term: p^2
    p2 = np.matmul(p, p)

    # Commutator term [b^†, b] = -Z
    Z = np.array([[1, 0], [0, -1]])  # Pauli Z matrix for fermion number
    commutator_term = np.kron(Z, W_double_prime)

    # Construct the block-diagonal kinetic term (bosonic and fermionic parts)
    # Bosonic part is the same for both, hence we use kron with the identity matrix
    kinetic_term = np.kron(I_f, p2)

    # Potential term (W' contribution)
    potential_term = np.kron(I_f, np.matmul(W_prime, W_prime))

    # Construct the full Hamiltonian
    H_SQM = 0.5 * (kinetic_term + potential_term + commutator_term)
    H_SQM[np.abs(H_SQM) < 10e-12] = 0
    
    return H_SQM

In [363]:
# Example usage for a 4x4 matrix
cut_off = 16
potential = 'QHO'
H = calculate_Hamiltonian(cut_off, potential)
#Hamiltonian = qml.pauli_decompose(H)
hamiltonian = SparsePauliOp.from_operator(H)

num_qubits = hamiltonian.num_qubits

In [364]:
num_qubits

5

In [324]:
# Set up BasicEntanglerLayers ansatz
num_layers = 2
params_shape = BasicEntanglerLayers.shape(n_layers=num_layers, n_wires=num_qubits)
params = 1.0*np.pi * pnp.random.random(size=params_shape)
params = (2.0 * pnp.random.random(size=params_shape) - 1.0) * pnp.pi/2
#params = pnp.random.uniform(-0.5, 0.5, size=params_shape)

In [365]:
def ry_cnot_ansatz(params):
    for i in range(num_qubits):
        qml.RY(params[i], wires=i)  # Apply RY rotation on each qubit

    #CNOT between qubit 0 and the last qubit
    qml.CNOT(wires=[0, num_qubits - 1])

In [366]:
params = pnp.random.randn(num_qubits, requires_grad=True)
params

tensor([ 0.32031806, -1.24856189,  0.19352759,  1.95559378, -0.83342209], requires_grad=True)

In [367]:
# Device
dev = qml.device('default.qubit', wires=num_qubits)

# Define the cost function with qml.Hermitian
@qml.qnode(dev)
def cost_function(params):
    #BasicEntanglerLayers(params, wires=range(num_qubits))
    ry_cnot_ansatz(params)
    return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))

In [368]:
# Initialize the optimizer and parameters
#optimizer = GradientDescentOptimizer(stepsize=0.25)
optimizer = AdamOptimizer(stepsize=0.25)

In [369]:
# Run the VQE optimization loop
max_iterations = 2500
convergence_tol = 1e-9
for i in range(max_iterations):
    params, energy = optimizer.step_and_cost(cost_function, params)
    
    # Print the energy every few steps
    if i % 10 == 0:
        print(f"Iteration {i}: Energy = {energy}")

    # Check for convergence
    if np.abs(energy) < convergence_tol:
        print(f"Converged at iteration {i}")
        break

print("Final optimized energy:", energy)
print("Optimized parameters:", params)

Iteration 0: Energy = 5.298279940949306
Iteration 10: Energy = 1.4310244071996836
Iteration 20: Energy = 0.11880092787249848
Iteration 30: Energy = 0.1263020988613654
Iteration 40: Energy = 0.03914422174693395
Iteration 50: Energy = 0.003851016284949776
Iteration 60: Energy = 0.0008330423051915726
Iteration 70: Energy = 0.0013946769549856603
Iteration 80: Energy = 0.0008943639629241138
Iteration 90: Energy = 0.00042475230432344527
Iteration 100: Energy = 0.00014881389286677658
Iteration 110: Energy = 3.378579202975488e-05
Iteration 120: Energy = 1.6064219092485006e-05
Iteration 130: Energy = 7.802738175893318e-06
Iteration 140: Energy = 2.0504375440100836e-06
Iteration 150: Energy = 8.640065279774611e-07
Iteration 160: Energy = 4.5493850297482824e-08
Iteration 170: Energy = 7.565959603193565e-08
Iteration 180: Energy = 3.7071629913717674e-08
Iteration 190: Energy = 9.57000160712838e-09
Iteration 200: Energy = 2.2129854566707344e-09
Converged at iteration 201
Final optimized energy: 7.6